# Nieuwe methode proberen

Do your imports

In [1]:
from pathlib import Path
import geopandas as gpd
import folium 
from shapely.geometry import LineString, Polygon, box
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import shape
import geopandas as gpd
import pandas as pd
from pathlib import Path
import pickle
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.network_config_data import (
    HazardSection,
    NetworkConfigData,
    NetworkSection,
    OriginsDestinationsSection
)

#RA2CE IMPORTS
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection,)
from ra2ce.network.network_wrappers.osm_network_wrapper.osm_network_wrapper import (OsmNetworkWrapper,)
from ra2ce.network.exporters.geodataframe_network_exporter import GeoDataFrameNetworkExporter
from ra2ce.network.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter
from ra2ce.ra2ce_handler import Ra2ceHandler

# from ra2ce.configuration.config_wrapper import ConfigWrapper
# from ra2ce.analysis.analysis_config_wrapper import AnalysisConfigWrapper
# from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisConfigData

# from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisSectionBase, AnalysisSectionLosses, ProjectSection
# from ra2ce.analysis.analysis_config_data.enums.analysis_damages_enum import AnalysisDamagesEnum
# from ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum import AnalysisLossesEnum
# from ra2ce.analysis.analysis_config_data.enums.damage_curve_enum import DamageCurveEnum
# from ra2ce.analysis.analysis_config_data.enums.event_type_enum import EventTypeEnum
# from ra2ce.analysis.analysis_config_data.enums.weighing_enum import WeighingEnum
# from ra2ce.analysis.losses.multi_link_origin_closest_destination import MultiLinkOriginClosestDestination
# from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
# from ra2ce.network.network_config_wrapper import NetworkConfigWrapper
# from ra2ce.network.graph_files.graph_files_collection import GraphFilesCollection
# from ra2ce.network.graph_files.graph_file import GraphFile
# from ra2ce.network.graph_files.network_file import NetworkFile
# from ra2ce.network.hazard.hazard_names import HazardNames
# import ra2ce.network.networks_utils as nut


c:\Users\meije_le\AppData\Local\miniforge3\envs\ra2ce_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# specify the name of the path to the project folder where you created the RA2CE folder setup

root_dir = Path(r'C:\python\powerpath\data')
assert root_dir.exists()
static_path = root_dir.joinpath("static")
hazard_path = static_path.joinpath("hazard")
network_path = static_path.joinpath("network")
output_path = static_path.joinpath("output_graph")


# Data pre-processing

In [5]:
# some preliminary functions

def get_all_files(directory: str) -> list[Path]:
    p = Path(directory)
    return [file for file in p.iterdir() if file.is_file()]

def read_pickle(file_path: str):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

def read_gpkg_to_gdf(file_path: str, layer: str = None) -> gpd.GeoDataFrame:
    # Read the geopackage file into a GeoDataFrame
    gdf = gpd.read_file(file_path, layer=layer)
    return gdf

In [6]:
hazard_files = get_all_files(hazard_path)
hazard_crs = "EPSG:4326" # for the hackathon case => "EPSG:4326" 

for hazard_file in hazard_files:
    print (hazard_file)

C:\python\powerpath\data\static\hazard\clipped_hazard_map.tif
C:\python\powerpath\data\static\hazard\clipped_hazard_map.tif.aux.xml
C:\python\powerpath\data\static\hazard\delfland_ghg200m_wgs84.tif


# First check if there are values below 0 and if so process it

In [7]:
import rasterio
import numpy as np
from pathlib import Path

# Function to process each raster file
def process_raster(hazard_file, output_dir):
    with rasterio.open(hazard_file) as src:
        # Read the data
        data = src.read(1)
        
        # Check if there are any values below zero
        if not np.any(data < 0):
            print(f"There are no no data values in {hazard_file.name}")
            return  # Skip processing
        
        # Create a mask for values below zero
        mask = data < 0
        
        # Set values below zero to NoData
        data[mask] = src.nodata
        
        # Update the metadata to reflect changes
        meta = src.meta.copy()
        
        # Write the modified data to a new file in the output directory
        output_file = output_dir / f"{hazard_file.stem}_processed.tif"
        with rasterio.open(output_file, 'w', **meta) as dst:
            dst.write(data, 1)
        
        print(f"Processed {hazard_file} to {output_file}")



In [ ]:
hazard_path_processed = hazard_path.joinpath("processed")
for hazard_file in hazard_files:
    process_raster(hazard_file, hazard_path_processed)

In [11]:
hazard_files_processed = get_all_files(hazard_path_processed)
hazard_crs = "EPSG:4326" # for the hackathon case => "EPSG:4326" 

for hazard_file in hazard_files_processed:
    print (hazard_file)

C:\python\powerpath\data\static\hazard\processed\delfland_ghg200m_wgs84_processed.tif
